In [125]:
import os, math
import pandas as pd
import pandapower as pp
import numpy as np

data_dir = 'Modified_116_LV_CSV'

In [126]:
# Source data
source_df = pd.read_csv(os.path.join(data_dir,'Source.csv'), skiprows=1, sep='=')
source_dict = {i:float(row.iloc[0].split()[0]) for i, row in source_df.iterrows()}
print(source_dict)

source_dict['ISC1'] = source_dict['ISC1'] / 1000
source_dict['ISC3'] = source_dict['ISC3'] / 1000

# Transformer data
trafo_df = pd.read_csv(os.path.join(data_dir,'Transformer.csv'), skiprows=1, sep=';')
trafo_dict = trafo_df.iloc[0].to_dict()
print(trafo_dict)

{'Voltage': 11.0, 'pu': 1.05, 'ISC3': 3000.0, 'ISC1': 5.0}
{'Name': 'TR1', ' phases': 3, ' bus1': 'SourceBus', ' bus2': 1, ' kV_pri': 11, ' kV_sec': 0.416, ' MVA': 0.8, ' Conn_pri': ' Delta', ' Conn_sec': ' Wye', ' %XHL': 4, '% resistance': 0.4}


In [127]:
s_sc_mva = np.sqrt(3) * source_dict['Voltage'] * source_dict['ISC3'] #MVA

net = pp.create_empty_network()

bus_map = {}

hv_bus = pp.create_bus(net, name=trafo_dict[' bus1'], vn_kv=source_dict['Voltage'], type="b")
lv_bus = pp.create_bus(net, name=trafo_dict[' bus2'], vn_kv=trafo_dict[' kV_sec'], type="b")

bus_map[trafo_dict[' bus1']] = hv_bus
bus_map[trafo_dict[' bus2']] = lv_bus

pp.create_ext_grid(
    net, bus=hv_bus, vm_pu=source_dict['pu'],
    s_sc_max_mva=s_sc_mva,
    rx_max=0.1, rx_min=None,
    # max_p_mw=None, min_p_mw=None,
    # max_q_mvar=None, min_q_mvar=None, index=None,
    r0x0_max=0.1, x0x_max=1.0
)

pp.create_transformer_from_parameters(
    net=net, hv_bus=hv_bus, lv_bus=lv_bus,
    sn_mva=trafo_dict[' MVA'],
    vn_hv_kv=trafo_dict[' kV_pri'],
    vn_lv_kv=trafo_dict[' kV_sec'],
    vk_percent=trafo_dict[' %XHL'],
    vkr_percent=trafo_dict['% resistance'],
    pfe_kw=0.0, i0_percent=0.0, #shift_degree=0.0, 
    name=trafo_dict['Name']
)

pp.create_shunt(
    net, bus=lv_bus,
    q_mvar=-0.01, p_mw=10e-3,
    vn_kv=trafo_dict[' kV_sec'],
    name="lv_shunt"
)

np.int64(0)

In [128]:
lines_df = pd.read_excel(os.path.join(data_dir, "Lines.xlsx"), skiprows=1)
lines_df['Length'] = lines_df['Length'] / 1000 # m to km
lines_df['Units'] = 'km'
lines_df.head()

lineCodes_df = pd.read_csv(os.path.join(data_dir, "LineCodes.csv"), skiprows=1, sep=';')

all_bus_ids = np.unique(lines_df[['Bus1', 'Bus2']].values.ravel('K'))
for id in all_bus_ids:
    id = math.floor(id)
    if id not in bus_map.keys():
        bus = pp.create_bus(net, name=id, vn_kv=trafo_dict[' kV_sec'], type="b")
        bus_map[id] = bus

In [129]:
lineCodes_df['max_i_ka'] = np.ones(len(lineCodes_df))

In [130]:
full_line_df = lines_df.merge(lineCodes_df, left_on="LineCode", right_on="Name", how="left")
# full_line_df.head(10)

net.line.drop(net.line.index, inplace=True)
for _, line in full_line_df.iterrows():
    # print(line)

    if (line['C0'] == 0): line['C0'] = 200 # nF/km
    if (line['C1'] == 0): line['C1'] = 200 # nF/km
    
    pp.create_line_from_parameters(
        net, from_bus = bus_map[math.floor(line['Bus1'])],
        to_bus = bus_map[math.floor(line['Bus2'])],
        length_km = line['Length'],
        r_ohm_per_km=line["R1"], r0_ohm_per_km=line["R0"],
        x_ohm_per_km=line["X1"], x0_ohm_per_km=line["X0"],
        c_nf_per_km=line["C1"], c0_nf_per_km=line["C0"],
        max_i_ka=line["max_i_ka"],
        name=line["Name_x"]
    )

In [133]:
suspicious_lines = net.line[
    (np.isclose(net.line["length_km"], 0, rtol=1e-2)) |
    (np.isclose(net.line["r_ohm_per_km"], 0, rtol=1e-2)) |
    (np.isclose(net.line["x_ohm_per_km"], 0, rtol=1e-2))
]
suspicious_lines

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo,r0_ohm_per_km,x0_ohm_per_km,c0_nf_per_km,g0_us_per_km


In [134]:
net.trafo["vector_group"] = 'Dyn'
net.trafo["vk0_percent"] = net.trafo["vk_percent"]
net.trafo["vkr0_percent"] = net.trafo["vkr_percent"]
net.trafo["mag0_percent"] = 100
net.trafo["mag0_rx"] = 0
net.trafo["si0_hv_partial"] = 0.9

In [168]:
import pandas as pd

def export_results(file_path, net, init='auto', max_iteration=100, tolerance_mva=1e-8):
    """
    Export all result DataFrames (attributes of net starting with 'res_') to an Excel file.
    
    Parameters:
    - file_path: str, path to the Excel file to save the results.
    - net: object containing result DataFrames as attributes starting with 'res_'.
    """

    try:
        pp.runpp_3ph(
            net, init=init,
            max_iteration=max_iteration,
            tolerance_mva=tolerance_mva,
            calc_voltage_angles=True,
            v_debug=True
        )
    except Exception as e:
        return e  # or raise e if you want the exception to propagate

    with pd.ExcelWriter(file_path) as writer:
        for attr in dir(net):
            if attr.startswith("res_"):
                df = getattr(net, attr)
                if isinstance(df, pd.DataFrame) and not df.empty:
                    sheet_name = attr[:31]  # Excel sheet names must be <= 31 characters
                    df.to_excel(writer, sheet_name=sheet_name)
    return True

def debug_result(net, init='auto', max_iteration=100, tolerance_mva=1e-8):
    try:
        pp.runpp_3ph(
            net, init=init,
            max_iteration=max_iteration,
            tolerance_mva=tolerance_mva,
            calc_voltage_angles=True,
            v_debug=True
        )
    except Exception as e:
        return False
    return True

In [ ]:
# Create Loads
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
# print(loads_df.head(10))
sample_loads = loads_df.sample(15)

net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
load_map = {}
for _, row in sample_loads.iterrows():
    # print(_, '->', row)
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0
    if (bus_id in bus_map) & (row['phases'] in {'A', 'B', 'C'}):
        if row['phases']=='A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases']=='B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else: # row['phases']=='C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))
        load = pp.create_asymmetric_load(
            net, bus=bus_map[bus_id],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
        load_map[load] = row['Yearly']

print(net.asymmetric_load)
export_results("batch_network_results.xlsx", net)

pandapower.auxiliary.LoadflowNotConverged('Power Flow nr did not converge after 100 iterations!')

In [180]:
N = 500
batch = list(range(5, 51, 5))
for b in batch:
    success = 0
    for i in range(N):
        sample_loads = loads_df.sample(b)

        net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
        load_map = {}
        for _, row in sample_loads.iterrows():
            # print(_, '->', row)
            bus_id = math.floor(row['Bus'])
            pa = qa = pb = qb = pc = qc = 0
            if (bus_id in bus_map) & (row['phases'] in {'A', 'B', 'C'}):
                if row['phases']=='A':
                    pa = row['kW']
                    qa = row['kW'] * np.tan(np.arccos(row['PF']))
                elif row['phases']=='B':
                    pb = row['kW']
                    qb = row['kW'] * np.tan(np.arccos(row['PF']))
                else: # row['phases']=='C'
                    pc = row['kW']
                    qc = row['kW'] * np.tan(np.arccos(row['PF']))
                load = pp.create_asymmetric_load(
                    net, bus=bus_map[bus_id],
                    p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
                    p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
                    p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
                    name=row['Name']
                )
                load_map[load] = row['Yearly']

        if debug_result(net): success += 1
    print(f"[{b}]\tConvergence rate = {100 * success / N} %")

[5]	Convergence rate = 99.6 %


c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pypower\newtonpf.py:441: MatrixRankWarning: Matrix is exactly singular
  dx = -1 * spsolve(J, F, permc_spec=permc_spec, use_umfpack=use_umfpack)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\create_jacobian.py:27: RuntimeWarning: invalid value encountered in divide
  dVm_x, dVa_x = dSbus_dV_numba_sparse(Ybus.data, Ybus.indptr, Ybus.indices, V, V / abs(V), Ibus)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:468: RuntimeWarning: invalid value encountered in divide
  i_abc_it_wye = (np.divide(s_abc_pu, v_abc_it)).conjugate()
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:469: RuntimeWarning: invalid value encountered in divide
  i_abc_it_delta = np.matmul(i_del_xfmn, (np.divide(s_abc_delta_pu, np.matmul
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib

[10]	Convergence rate = 83.6 %
[15]	Convergence rate = 66.4 %


c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pypower\newtonpf.py:441: MatrixRankWarning: Matrix is exactly singular
  dx = -1 * spsolve(J, F, permc_spec=permc_spec, use_umfpack=use_umfpack)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\create_jacobian.py:27: RuntimeWarning: invalid value encountered in divide
  dVm_x, dVa_x = dSbus_dV_numba_sparse(Ybus.data, Ybus.indptr, Ybus.indices, V, V / abs(V), Ibus)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:468: RuntimeWarning: invalid value encountered in divide
  i_abc_it_wye = (np.divide(s_abc_pu, v_abc_it)).conjugate()
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:469: RuntimeWarning: invalid value encountered in divide
  i_abc_it_delta = np.matmul(i_del_xfmn, (np.divide(s_abc_delta_pu, np.matmul
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib

[20]	Convergence rate = 32.6 %


c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\create_jacobian.py:27: RuntimeWarning: invalid value encountered in divide
  dVm_x, dVa_x = dSbus_dV_numba_sparse(Ybus.data, Ybus.indptr, Ybus.indices, V, V / abs(V), Ibus)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pypower\newtonpf.py:441: MatrixRankWarning: Matrix is exactly singular
  dx = -1 * spsolve(J, F, permc_spec=permc_spec, use_umfpack=use_umfpack)
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:468: RuntimeWarning: invalid value encountered in divide
  i_abc_it_wye = (np.divide(s_abc_pu, v_abc_it)).conjugate()
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandapower\pf\runpp_3ph.py:469: RuntimeWarning: invalid value encountered in divide
  i_abc_it_delta = np.matmul(i_del_xfmn, (np.divide(s_abc_delta_pu, np.matmul
c:\Users\vitor\AppData\Local\Programs\Python\Python312\Lib

[25]	Convergence rate = 31.2 %
[30]	Convergence rate = 13.8 %
[35]	Convergence rate = 8.0 %
[40]	Convergence rate = 3.8 %
[45]	Convergence rate = 4.2 %
[50]	Convergence rate = 0.0 %


In [170]:
# Load by load:
load_problem = []

for _, row in loads_df.iterrows():
    net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
    
    # print(_, '->', row)
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0
    if (bus_id in bus_map) & (row['phases'] in {'A', 'B', 'C'}):
        if row['phases']=='A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases']=='B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else: # row['phases']=='C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))
        load = pp.create_asymmetric_load(
            net, bus=bus_map[bus_id],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
    if not debug_result(net): load_problem.append(row['Name'])

load_problem

[]

In [172]:
a = False
if not a: print(a)

False


In [ ]:
# Create Loads
loads_df = pd.read_excel(os.path.join(data_dir, "Loads.xlsx"), skiprows=2)
# print(loads_df.head(10))

net.asymmetric_load.drop(net.asymmetric_load.index, inplace=True)
load_map = {}
for _, row in loads_df.iterrows():
    # print(_, '->', row)
    bus_id = math.floor(row['Bus'])
    pa = qa = pb = qb = pc = qc = 0
    if (bus_id in bus_map) & (row['phases'] in {'A', 'B', 'C'}):
        if row['phases']=='A':
            pa = row['kW']
            qa = row['kW'] * np.tan(np.arccos(row['PF']))
        elif row['phases']=='B':
            pb = row['kW']
            qb = row['kW'] * np.tan(np.arccos(row['PF']))
        else: # row['phases']=='C'
            pc = row['kW']
            qc = row['kW'] * np.tan(np.arccos(row['PF']))
        load = pp.create_asymmetric_load(
            net, bus=bus_map[bus_id],
            p_a_mw=pa / 1000, q_a_mvar=qa / 1000,
            p_b_mw=pb / 1000, q_b_mvar=qb / 1000,
            p_c_mw=pc / 1000, q_c_mvar=qc / 1000,
            name=row['Name']
        )
        load_map[load] = row['Yearly']

print(len(net.asymmetric_load))
export_results("network_results.xlsx", net, max_iteration=1000)

,name,bus,p_a_mw,q_a_mvar,p_b_mw,q_b_mvar,p_c_mw,q_c_mvar,sn_mva,scaling,in_service,type
0,LOAD1,5,0.001,0.000329,0.000,0.000000,0.000,0.000000,NaN,1.0,True,wye
1,LOAD2,7,0.000,0.000000,0.001,0.000329,0.000,0.000000,NaN,1.0,True,wye
2,LOAD3,10,0.001,0.000329,0.000,0.000000,0.000,0.000000,NaN,1.0,True,wye
3,LOAD4,11,0.001,0.000329,0.000,0.000000,0.000,0.000000,NaN,1.0,True,wye
4,LOAD5,12,0.001,0.000329,0.000,0.000000,0.000,0.000000,NaN,1.0,True,wye
5,LOAD6,13,0.000,0.000000,0.001,0.000329,0.000,0.000000,NaN,1.0,True,wye
6,LOAD7,21,0.000,0.000000,0.001,0.000329,0.000,0.000000,NaN,1.0,True,wye
7,LOAD8,25,0.000,0.000000,0.000,0.000000,0.001,0.000329,NaN,1.0,True,wye
8,LOAD9,26,0.001,0.000329,0.000,0.000000,0.000,0.000000,NaN,1.0,True,wye
9,LOAD10,29,0.000,0.000000,0.001,0.000329,0.000,0.000000,NaN,1.0,True,wye
